In [1]:
!pip install transformers dataset


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import load_dataset
import torch

c:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
raw_dataset = load_dataset("wmt14", "de-en", split='train[:1]')

In [4]:
raw_dataset

Dataset({
    features: ['translation'],
    num_rows: 1
})

In [5]:
raw_dataset["translation"][0]

{'de': 'Wiederaufnahme der Sitzungsperiode', 'en': 'Resumption of the session'}

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")

def preprocess_function(examples):
    inputs = ["translate English to German: " + ex['en'] for ex in examples['translation']]
    targets = [ex['de'] for ex in examples['translation']]

    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = tokenizer(targets, max_length=128, truncation=True, padding="max_length").input_ids

    return model_inputs

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [7]:
tokenized_dataset = raw_dataset.map(preprocess_function, batched=True)

In [8]:
model = T5ForConditionalGeneration.from_pretrained("t5-small")

In [14]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    push_to_hub=False,
    report_to=[]
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

trainer.train()

Step,Training Loss


TrainOutput(global_step=2, training_loss=15.96501350402832, metrics={'train_runtime': 1.9993, 'train_samples_per_second': 1.0, 'train_steps_per_second': 1.0, 'total_flos': 67670900736.0, 'train_loss': 15.96501350402832, 'epoch': 2.0})

In [ ]:
text = "translate English to German: I like you"
input_ids = tokenizer(text, return_tensors="pt").input_ids.to(model.device)
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Ich muß Ihnen sagen, daß Sie diesbezüglich recht gut kennen.


In [11]:
model.save_pretrained('./t5_translation_model')